In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import json

from tqdm import tqdm_notebook
from glob import glob
from pathlib import Path
#from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img, image

%matplotlib inline                               

In [15]:
pklot_path_list = list(Path('.').glob('PKLot/PKLot/*/*/*/*.jpg'))
json_path_list = list(Path('.').glob('json/*.json'))

In [16]:
pklot_df = pd.DataFrame({'image_path': pklot_path_list})
json_df = pd.DataFrame({'json_path': json_path_list})

In [17]:
pklot_df['date_id'] = pklot_df['image_path'].map(lambda x: x.stem)
json_df['date_id'] = json_df['json_path'].map(lambda x: x.stem)

In [19]:
new_df = pklot_df.merge(json_df, left_on='date_id', right_on='date_id', how='inner')

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img(pklot_files[0], target_size=(150, 150))  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 4 == 0:
        break

In [31]:
def jsonToList(pklot_file, json_file):
    data_list = []

    with open(json_file, 'r') as json_data:
        for line in json_data:
            row = json.loads(line)
            for segmented_data in row['spaces']:
                segmented_data['image_path'] = pklot_file
                segmented_data['json_path'] = json_file
                segmented_data['date_id'] = pklot_file.stem
                segmented_data['rotated_rect_degree'] = segmented_data['rotatedRect']['angle']['d']
                segmented_data['rotated_rect_center_x'] = segmented_data['rotatedRect']['center']['x']
                segmented_data['rotated_rect_center_y'] = segmented_data['rotatedRect']['center']['y']
                segmented_data['rotated_rect_height'] = segmented_data['rotatedRect']['size']['h']
                segmented_data['rotated_rect_width'] = segmented_data['rotatedRect']['size']['w']
                
                segmented_data.pop('rotatedRect')
                data_list.append(segmented_data)
    return data_list

In [32]:
data = []

for pklot_path, json_path in tqdm_notebook(zip(new_df['image_path'], new_df['json_path']), total=12417):
    data.extend(jsonToList(pklot_path, json_path))

In [34]:
pklot_df = pd.DataFrame(data)

In [38]:
pklot_df.rename(columns={'id': 'parking_id'}, inplace=True)

In [39]:
pklot_df.head()

,contour,date_id,parking_id,image_path,json_path,occupied,rotated_rect_center_x,rotated_rect_center_y,rotated_rect_degree,rotated_rect_height,rotated_rect_width
0,"[{'x': '641', 'y': '570'}, {'x': '726', 'y': '...",2013-01-18_18_35_15,1,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,json/2013-01-18_18_35_15.json,0,651,636,-34,141,83
1,"[{'x': '698', 'y': '505'}, {'x': '782', 'y': '...",2013-01-18_18_35_15,2,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,json/2013-01-18_18_35_15.json,1,705,563,-44,139,77
2,"[{'x': '719', 'y': '427'}, {'x': '799', 'y': '...",2013-01-18_18_35_15,3,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,json/2013-01-18_18_35_15.json,1,732,480,-48,124,60
3,"[{'x': '754', 'y': '367'}, {'x': '824', 'y': '...",2013-01-18_18_35_15,4,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,json/2013-01-18_18_35_15.json,0,762,409,-51,113,54
4,"[{'x': '767', 'y': '313'}, {'x': '833', 'y': '...",2013-01-18_18_35_15,5,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,json/2013-01-18_18_35_15.json,0,773,346,-56,109,49


In [40]:
pklot_df.to_csv('pklot_formatted_data.csv', index=False)